# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import os
import re

import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl,\
                                StringType as Str, IntegerType as Int, DateType as Date,\
                                TimestampType as TimeStamp

In [2]:
# full immigration data
os.listdir("../../../data/18-83510-I94-Data-2016/")

['i94_apr16_sub.sas7bdat',
 'i94_sep16_sub.sas7bdat',
 'i94_nov16_sub.sas7bdat',
 'i94_mar16_sub.sas7bdat',
 'i94_jun16_sub.sas7bdat',
 'i94_aug16_sub.sas7bdat',
 'i94_may16_sub.sas7bdat',
 'i94_jan16_sub.sas7bdat',
 'i94_oct16_sub.sas7bdat',
 'i94_jul16_sub.sas7bdat',
 'i94_feb16_sub.sas7bdat',
 'i94_dec16_sub.sas7bdat']

In [3]:
import os
import glob

In [6]:
files = glob.glob(os.path.join("../src", "*.py"))
files

['../src/upload_raw_data.py']

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

The following datasets wil be used for the project:
* I94 Immigration Data
* U.S. City Demographic Data

In addition, a list of i94 port codes to their respective US city and state names will be extracted from the I94 Immigration Data Dictionary.

In [3]:
# Reading the immigration data sample
imm_data_sample = pd.read_csv("../sample_data/immigration_data_sample.csv")
imm_data_sample.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [4]:
# Reading the us cities demographics data
us_cities_demog = pd.read_csv("../sample_data/us-cities-demographics.csv")
us_cities_demog.head()

,City;State;Median Age;Male Population;Female Population;Total Population;Number of Veterans;Foreign-born;Average Household Size;State Code;Race;Count
0,Silver Spring;Maryland;33.8;40601;41862;82463;...
1,Quincy;Massachusetts;41.0;44129;49500;93629;41...
2,Hoover;Alabama;38.5;38040;46799;84839;4819;822...
3,Rancho Cucamonga;California;34.5;88127;87105;1...
4,Newark;New Jersey;34.6;138040;143873;281913;58...


In [ ]:
df.withColumn("yearTmp", df.year.cast(IntegerType))
    .drop("year")

In [6]:
spark = SparkSession.builder.\
        config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
        .enableHiveSupport().getOrCreate()

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [7]:
imm_data_sample.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


* i94yr: year
* i94mon: month
* i94cit, i94res: country code (both valid and invalid)
* i94port: us port code (both valid and invalid)
* arrdate: arrival date
* i94mode: mode of arrival
* i94addr: U.S. state address
* depdate: departure date
* i94bir: respondent's age
* i94visa: respondent's visa type
* count: used for summary statistics
* dtadfield: date added to I-94 files
* visapost: department of state where visa is issued
* occup: occupation in U.S.
* entdepa: arrival flag - admitted or paroled into the U.S.
* entdepd: departure flag - departed, lost I-94 or is deceased
* entdepu: update flag - either apprehended, overstayed, adjusted to perm residence
* matflag: match flag - match of arrival and departure records
* biryear: birth year
* dtaddto: date to which admitted to U.S. (allowed to stay until)
* gender: non-immigrant gender
* insnum: ins num
* airline: airline used to arrive in U.S.
* adnum: admission number
* fltno: flight number
* visatype: class of admission legally admitting the non-immigrant to temporarily stay in U.S.

In [8]:
imm_data_sample.isnull().mean()

Unnamed: 0    0.000
cicid         0.000
i94yr         0.000
i94mon        0.000
i94cit        0.000
i94res        0.000
i94port       0.000
arrdate       0.000
i94mode       0.000
i94addr       0.059
depdate       0.049
i94bir        0.000
i94visa       0.000
count         0.000
dtadfile      0.000
visapost      0.618
occup         0.996
entdepa       0.000
entdepd       0.046
entdepu       1.000
matflag       0.046
biryear       0.000
dtaddto       0.000
gender        0.141
insnum        0.965
airline       0.033
admnum        0.000
fltno         0.008
visatype      0.000
dtype: float64

Cleaning steps:
* Can read i94year as INT

In [9]:
imm_data_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [10]:
imm_data_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

### Read in US Cities Demographics Dataset and clean it

In [11]:
us_cities_demog.head()

,City;State;Median Age;Male Population;Female Population;Total Population;Number of Veterans;Foreign-born;Average Household Size;State Code;Race;Count
0,Silver Spring;Maryland;33.8;40601;41862;82463;...
1,Quincy;Massachusetts;41.0;44129;49500;93629;41...
2,Hoover;Alabama;38.5;38040;46799;84839;4819;822...
3,Rancho Cucamonga;California;34.5;88127;87105;1...
4,Newark;New Jersey;34.6;138040;143873;281913;58...


In [12]:
def clean_cities_table(cities_df):
    new_cities_df = cities_df.copy()
    original_col = new_cities_df.columns[0]
    columns = cities_df.columns[0].split(";")

    def return_single_col_data(str_of_values, n_col):
        return str_of_values.split(";")[n_col]

    for i, col in enumerate(columns):
        new_cities_df[col] = new_cities_df[original_col].apply(return_single_col_data, n_col=i)

    new_cities_df.drop(columns=[original_col], inplace=True)
    
    return new_cities_df

In [13]:
new_us_cities_demog = clean_cities_table(us_cities_demog)
new_us_cities_demog.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.6,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402


In [14]:
new_us_cities_demog.isnull().mean()

City                      0.0
State                     0.0
Median Age                0.0
Male Population           0.0
Female Population         0.0
Total Population          0.0
Number of Veterans        0.0
Foreign-born              0.0
Average Household Size    0.0
State Code                0.0
Race                      0.0
Count                     0.0
dtype: float64

In [18]:
new_us_cities_demog.to_csv("../sample_data/new_us_cities_demographics.csv", index=False)

In [19]:
cities_schema = R([
        Fld("City", Str()),
        Fld("State", Str()),
        Fld("Median Age", Dbl()),
        Fld("Male Population", Int()),
        Fld("Female Population", Int()),
        Fld("Total Population", Int()),
        Fld("Number of Veterans", Int()),
        Fld("Foreign-born", Int()),
        Fld("Average Household Size", Dbl()),
        Fld("State Code", Str()),
        Fld("Race", Str()),
        Fld("Count", Int())
    ])

us_cities_demog_spark = spark.read.load("../sample_data/new_us_cities_demographics.csv",
                                        format="csv", sep=":", schema=cities_schema, header="true")

In [20]:
us_cities_demog_spark.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)



### Extract Map of i94 Port Codes to US Cities and States

In [21]:
def extract_city_and_state(city_state_str):
    if "," in city_state_str:
        city, state = city_state_str.split(",")[0].strip(),\
                        city_state_str.split(",")[1].strip()
    else:
        city = city_state_str
        state = np.nan
    return city, state

def extract_ports_and_cities(file):
    processing_i94port = False

    i94port_list = []
    city_list = []
    state_list = []
    
    unwanted_chars = r'[^a-zA-Z0-9(), ]'

    for line in txt_file:
        if "I94PORT" in line:
            processing_i94port = True
        if processing_i94port and "=" in line:
            i94port, city_w_state = line.split("=")[0], line.split("=")[1]
            # remove unwanted characters from string
            i94port = re.sub(unwanted_chars, '', i94port)
            city_w_state = re.sub(unwanted_chars, '', city_w_state)
            city, state = extract_city_and_state(city_w_state)
            i94port_list.append(i94port)
            city_list.append(city)
            state_list.append(state)
        if processing_i94port and (";" in line):
            processing_i94port = False
            break
    return i94port_list, city_list, state_list

In [22]:
txt_file = open("../sample_data/I94_SAS_Labels_Descriptions.SAS", "r")
i94ports, cities, states = extract_ports_and_cities(txt_file)

data = {'i94 Port': i94ports, 'City': cities, 'States': states}
i94ports = pd.DataFrame (data, columns = ['i94 Port','City', 'States'])
i94ports.head()

,i94 Port,City,States
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK


In [23]:
i94ports.isnull().mean()

i94 Port    0.000000
City        0.000000
States      0.116667
dtype: float64

In [24]:
i94ports.to_csv("../sample_data/i94ports_to_cities.csv", index=False)

In [25]:
i94ports_schema = R([
        Fld("i94 Port", Str()),
        Fld("City", Str()),
        Fld("States", Str()),
    ])

i94ports_spark = spark.read.load("../sample_data/i94ports_to_cities.csv",
                                 format="csv", sep=":", schema=i94ports_schema, header="true")

In [26]:
i94ports_spark.printSchema()

root
 |-- i94 Port: string (nullable = true)
 |-- City: string (nullable = true)
 |-- States: string (nullable = true)



In [80]:
cities_1 = new_us_cities_demog.City.tolist()
cities_1 = set([city.lower() for city in cities_1])
cities_2 = set(i94ports.City.tolist())
cities_2 = set([city.lower() for city in cities_2])

In [86]:
len(cities_2)

634

In [85]:
common_cities = cities_1.intersection(cities_2)
common_cities
len(common_cities)

122

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

The Star Schema is chosen as the data model.

**Fact Table**:

Immigration
* immigration_id (int) Primary Key: ID for Immigration fact table.
* flight_num (int): The flight number of the visitor. Foreign key for the Flight dimension.
* visa_type (str): The US visa type or category held by the visitor. Foreign key for the Visa table.

**Dimension Tables**:

Flight
* flight_num (int) Primary key: ID for Flight table.
* airline (str): Airline taken by the visitor.

Visitor
* immigration_id (int) Primary key: ID for Visitor table.
* birth_year (int): The birth year of the visitor.
* occupation (str): The visitor's occupation in the US.
* country_of_residence (str): The visitor's country_of_residence.

Trip_Records
* immigration_id (int) Primary key: ID for Trip_Records table.
* arrival_date (int): The visitor's arrival date.
* year_of_arrival (int): Year of the visitor's arrival.
* month_of_arrival (int): Month of the visitor's arrival.
* country_of_prev_depart (str): The country the visitor departed from before arriving in the US.
* port_of_arrival_code (int): The I-94 port code for the visitor's port of arrival in the US.
* depart_date (int): The visitor's departure date.
* date_of_file_entry (int): Date the visitor's entry records were added to I-94 files.
* arrival_flag (str): Flag inidcating visitor is admitted or paroled into the U.S. .
* departure_flag (str): Flag inidcating visitor has departed, lost I-94 or is deceased.
* match_flag (str): Flag indicating there is a match of arrival and departure records.
* last_permitted_day_of_stay (int): Date of admission expiry for the visitor.
* state_of_visa_issued (str): Department of state where visa is issued to the visitor.

Visa
* visa_type (str) Primary key: Primary key. The US visa type or category held by the visitor, such as Wt, B2, etc.
* visa_category (str): The class of the visitor's visa: student, pleasure, business.

In [27]:
imm_data_sample.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

Raw data are staged in s3
Run ETL jobs on AWS EMR Spark clusters
Store fact and dimension tables in s3
Perform query with AWS Athena

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [28]:
output_data = "../output_data"

### Immigration Fact Table

In [29]:
# add immigration id column
imm_data_spark = imm_data_spark.withColumn("immigration_id", F.monotonically_increasing_id())

imm_fact_table_col = ["immigration_id", "fltno", "visatype"]

imm_fact_table = imm_data_spark.select(imm_fact_table_col)\
                    .dropDuplicates()\
                    .withColumnRenamed("fltno","flight_number")\
                    .withColumnRenamed("visatype","visa_type")

# write immigration fact table to parquet files
imm_fact_table.write.mode('overwrite').parquet(os.path.join(output_data, "immigration_fact.parquet"))

In [30]:
imm_fact_table = spark.read.parquet(os.path.join(output_data, "immigration_fact.parquet"))
imm_fact_table.show(5)

+--------------+-------------+---------+
|immigration_id|flight_number|visa_type|
+--------------+-------------+---------+
|            40|        00131|       WT|
|            64|        00731|       WT|
|           145|        00093|       WB|
|           203|        00410|       WT|
|           222|        00107|       WT|
+--------------+-------------+---------+
only showing top 5 rows



### Flight Dimension Table

In [60]:
flight_dim_table_col = ["fltno", "airline"]

flight_dim_table = imm_data_spark.select(flight_dim_table_col)\
                    .dropDuplicates()\
                    .withColumnRenamed("fltno","flight_number")\
                    .where(F.col("flight_number").isNotNull())

# write flight dimension table to parquet files
flight_dim_table.write.mode('overwrite').parquet(os.path.join(output_data, "flight_dimension.parquet"))

In [61]:
flight_dim_table = spark.read.parquet(os.path.join(output_data, "flight_dimension.parquet"))
flight_dim_table.show(5)

+-------------+-------+
|flight_number|airline|
+-------------+-------+
|        00683|     FI|
|          681|     FI|
|        00582|     DL|
|        00962|     AV|
|        01427|     UA|
+-------------+-------+
only showing top 5 rows



### Visitor Dimension Table

In [55]:
visitor_dim_table_col = ["immigration_id", "biryear", "occup", "i94res"]

visitor_dim_table = imm_data_spark.select(visitor_dim_table_col)\
                    .dropDuplicates()\
                    .withColumn("birth_year", imm_data_spark.biryear.cast(Int())).drop("biryear")\
                    .withColumnRenamed("occup","occupation")\
                    .withColumn("country_of_residence", imm_data_spark.i94res.cast(Int())).drop("i94res")

ordered_col = ["immigration_id", "birth_year", "occupation", "country_of_residence"]

visitor_dim_table = visitor_dim_table.select(ordered_col)

# write visitor dimension table to parquet files
visitor_dim_table.write.mode('overwrite').parquet(os.path.join(output_data, "visitor_dimension.parquet"))

In [56]:
visitor_dim_table = spark.read.parquet(os.path.join(output_data, "visitor_dimension.parquet"))
visitor_dim_table.show(5)

+--------------+----------+----------+--------------------+
|immigration_id|birth_year|occupation|country_of_residence|
+--------------+----------+----------+--------------------+
|           138|      1977|      null|                 103|
|           527|      1990|      null|                 103|
|           612|      1977|      null|                 438|
|           635|      1986|      null|                 103|
|          1066|      2010|      null|                 104|
+--------------+----------+----------+--------------------+
only showing top 5 rows



### Trip_Records Dimension Table

In [53]:
trip_records_dim_table_col = ["immigration_id", "arrdate", "i94yr", "i94mon", "i94cit",\
                              "i94port", "depdate", "dtadfile", "entdepa", "entdepd",\
                              "matflag", "dtaddto", "visapost"
                             ]

trip_records_dim_table = imm_data_spark.select(trip_records_dim_table_col)\
                            .dropDuplicates()\
                            .withColumn("arrival_date", imm_data_spark.arrdate.cast(Int())).drop("arrdate")\
                            .withColumn("year_of_arrival", imm_data_spark.i94yr.cast(Int())).drop("i94yr")\
                            .withColumn("month_of_arrival", imm_data_spark.i94mon.cast(Int())).drop("i94mon")\
                            .withColumn("country_of_prev_depart", imm_data_spark.i94cit.cast(Int())).drop("i94cit")\
                            .withColumnRenamed("i94port","port_of_arrival_code")\
                            .withColumn("depart_date", imm_data_spark.depdate.cast(Int())).drop("depdate")\
                            .withColumn("date_of_file_entry", imm_data_spark.dtadfile.cast(Int())).drop("dtadfile")\
                            .withColumnRenamed("entdepa","arrival_flag")\
                            .withColumnRenamed("entdepd","departure_flag")\
                            .withColumnRenamed("matflag","match_flag")\
                            .withColumn("last_permitted_day_of_stay", imm_data_spark.dtaddto.cast(Int())).drop("dtaddto")\
                            .withColumnRenamed("visapost","state_of_visa_issued")\

ordered_col = ["immigration_id", "arrival_date", "year_of_arrival", "month_of_arrival", "country_of_prev_depart",\
               "port_of_arrival_code", "depart_date", "date_of_file_entry", "arrival_flag", "departure_flag",\
               "match_flag", "last_permitted_day_of_stay", "state_of_visa_issued"
              ]

trip_records_dim_table = trip_records_dim_table.select(ordered_col)

# write visitor dimension table to parquet files
trip_records_dim_table.write.mode('overwrite').parquet(os.path.join(output_data, "trip_records_dimension.parquet"))

In [54]:
trip_records_dim_table = spark.read.parquet(os.path.join(output_data, "trip_records_dimension.parquet"))
trip_records_dim_table.show(5)

+--------------+------------+---------------+----------------+----------------------+--------------------+-----------+------------------+------------+--------------+----------+--------------------------+--------------------+
|immigration_id|arrival_date|year_of_arrival|month_of_arrival|country_of_prev_depart|port_of_arrival_code|depart_date|date_of_file_entry|arrival_flag|departure_flag|match_flag|last_permitted_day_of_stay|state_of_visa_issued|
+--------------+------------+---------------+----------------+----------------------+--------------------+-----------+------------------+------------+--------------+----------+--------------------------+--------------------+
|           316|       20545|           2016|               4|                   103|                 MIA|      20549|          20160401|           G|             O|         M|                   6292016|                null|
|           486|       20545|           2016|               4|                   103|               

### Visa Dimension Table

In [57]:
visa_dim_table_col = ["visatype", "i94visa"]

visa_dim_table = imm_data_spark.select(flight_dim_table_col)\
                    .dropDuplicates()\
                    .withColumnRenamed("visatype","visa_type")\
                    .withColumnRenamed("i94visa","visa_category")

visa_dim_table.write.mode('overwrite').parquet(os.path.join(output_data, "visa_dimension.parquet"))

In [58]:
visa_dim_table = spark.read.parquet(os.path.join(output_data, "visa_dimension.parquet"))
visa_dim_table.show(5)

+-----+-------+
|fltno|airline|
+-----+-------+
|00683|     FI|
|  681|     FI|
|00582|     DL|
|00962|     AV|
|01427|     UA|
+-----+-------+
only showing top 5 rows



#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

In [ ]:
# Check primary keys are not null

def check_

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.